# Variant Annotation: Synonymous vs Nonsynonymous

## What This Notebook Does

This notebook takes a list of genetic variants (mutations) and determines whether each variant:
- **Changes the protein** (nonsynonymous) - potentially affects function
- **Does NOT change the protein** (synonymous) - silent mutation
- **Is outside genes** (intergenic) - not in a protein-coding region

## Background: How DNA Encodes Proteins

```
DNA:     ATG  GCT  TAC  ...  (read in groups of 3 bases called "codons")
         ↓    ↓    ↓
Protein: Met  Ala  Tyr  ...  (each codon = 1 amino acid)
```

Because the genetic code is **redundant** (multiple codons can encode the same amino acid),
some mutations don't change the protein:

- `GCT` → Alanine
- `GCC` → Alanine (same amino acid! This would be a **synonymous** mutation)
- `GCT` → `GAT` → Aspartic acid (different amino acid = **nonsynonymous**)

## Inputs

1. **GenBank file** (dataset #13372): Contains the reference genome with gene annotations
2. **Variant table** (dataset #13437): List of mutations found in samples

---
## Step 1: Import Libraries

We need a few Python libraries:
- `gxy`: Galaxy-specific library to download/upload files
- `pandas`: For working with tables (like Excel but in Python)
- `Bio` (Biopython): For reading GenBank files and translating DNA
- `altair`: For creating interactive visualizations

In [ ]:
# Galaxy integration - lets us download datasets from the history
import gxy

# Pandas - work with data tables
import pandas as pd

# Biopython - parse GenBank files and translate DNA to protein
from Bio import SeqIO
from Bio.Data import CodonTable

# Altair - create interactive charts
import altair as alt
alt.data_transformers.disable_max_rows()  # Allow large datasets

# Suppress Altair/narwhals compatibility warnings (harmless but noisy)
import warnings
warnings.filterwarnings('ignore', message='.*narwhals.*')

print("Libraries loaded successfully!")

In [ ]:
# Define which Galaxy datasets to use
# These numbers correspond to the dataset IDs in your Galaxy history

GENBANK_DATASET = 13372    # GenBank annotation file (contains gene locations)
VARIANTS_DATASET = 13437   # Variant table (list of mutations)

---
## Step 2: Download Data from Galaxy

The `gxy.get()` function downloads a dataset from your Galaxy history to this notebook's environment.

In [ ]:
# Download the GenBank annotation file
genbank_path = await gxy.get(GENBANK_DATASET)
print(f"Downloaded GenBank file to: {genbank_path}")

In [ ]:
# Download the variant table
variants_path = await gxy.get(VARIANTS_DATASET)
print(f"Downloaded variants file to: {variants_path}")

---
## Step 3: Parse the GenBank File

### What is a GenBank file?

A GenBank file contains:
1. The full genome sequence (DNA)
2. Annotations showing where genes are located

Each gene has a "CDS" (Coding Sequence) feature that tells us:
- Start and end positions on the genome
- Gene name and protein product
- Which strand (+ or -) it's on

In [ ]:
# Load the GenBank file
record = SeqIO.read(genbank_path, "genbank")

print(f"Genome ID: {record.id}")
print(f"Genome length: {len(record.seq):,} bases")
print(f"Description: {record.description}")

In [ ]:
# Find all CDS (protein-coding) features
# These are the genes we'll use for annotation

cds_features = [f for f in record.features if f.type == "CDS"]

print(f"Found {len(cds_features)} protein-coding genes (CDS features)")
print("\nFirst few genes:")
for i, feat in enumerate(cds_features[:5]):
    # Get gene name from the 'gene' or 'locus_tag' field
    gene_name = feat.qualifiers.get("gene", feat.qualifiers.get("locus_tag", ["unknown"]))[0]
    product = feat.qualifiers.get("product", ["unknown"])[0]
    print(f"  {i+1}. {gene_name}: {product}")

---
## Step 4: Build a Lookup Table for Every Position in Every Gene

To annotate variants, we need to know for EVERY position in the genome:
- Is it inside a gene?
- If yes, which codon does it belong to?
- What amino acid does that codon encode?

We'll build a table where each row represents one nucleotide position.

### Understanding Codon Position

Each codon has 3 bases. The position within the codon matters:

```
Codon:    A   T   G
Position: 1   2   3
          ↑       ↑
          |       └── 3rd position: often "wobble" - mutations here are often synonymous
          └────────── 1st position: mutations here usually change the amino acid
```

In [ ]:
def get_gene_info(feature):
    """
    Extract useful information from a CDS feature.
    
    Returns a dictionary with gene name, product, and translation settings.
    """
    q = feature.qualifiers
    
    return {
        # Gene name - try 'gene' first, fall back to 'locus_tag'
        "gene": (q.get("gene") or q.get("locus_tag") or [""])[0],
        
        # Protein product description
        "product": (q.get("product") or [""])[0],
        
        # Codon start - which base of the first codon is the start
        # Usually 1, but can be 2 or 3 for partial genes
        "codon_start": int((q.get("codon_start") or ["1"])[0]),
        
        # Translation table - which genetic code to use
        # 1 = standard, 11 = bacterial/plastid, etc.
        "transl_table": int((q.get("transl_table") or ["1"])[0]),
    }

# Test on the first gene
test_info = get_gene_info(cds_features[0])
print("Example gene info:")
for key, value in test_info.items():
    print(f"  {key}: {value}")

In [ ]:
def codon_to_amino_acid(codon, transl_table=1):
    """
    Translate a 3-letter DNA codon to its amino acid.
    
    Examples:
        'ATG' -> 'M' (Methionine, start codon)
        'TAA' -> '*' (Stop codon)
        'GCT' -> 'A' (Alanine)
    """
    # Get the correct codon table (standard = 1, bacterial = 11, etc.)
    table = CodonTable.unambiguous_dna_by_id.get(transl_table, 
                                                  CodonTable.unambiguous_dna_by_id[1])
    
    codon = codon.upper()
    
    # Check if it's a stop codon
    if codon in table.stop_codons:
        return "*"
    
    # Look up the amino acid
    return table.forward_table.get(codon, "?")  # "?" if unknown

# Test the function
print("Testing codon translation:")
test_codons = ["ATG", "GCT", "GCC", "TAA", "TGA"]
for codon in test_codons:
    aa = codon_to_amino_acid(codon)
    print(f"  {codon} -> {aa}")

In [ ]:
def build_cds_lookup_table(record):
    """
    Build a table with one row per nucleotide for all CDS features.
    
    For each nucleotide, we record:
    - genome_pos: position on the genome (1-based)
    - gene: gene name
    - product: protein product
    - base: the nucleotide (A, T, G, or C)
    - codon: the 3-letter codon this base belongs to
    - codon_pos: position within the codon (1, 2, or 3)
    - aa: the amino acid the codon encodes
    """
    rows = []  # Will hold all our data
    
    # Process each CDS feature (gene)
    for feature in record.features:
        if feature.type != "CDS":
            continue
        
        # Get gene information
        info = get_gene_info(feature)
        
        # Extract the CDS sequence (handles strand automatically)
        cds_seq = str(feature.extract(record.seq))
        
        # Get the genomic positions for each base in the CDS
        # This handles + and - strand genes correctly
        strand = feature.location.strand or 1
        
        # Build list of genome positions in transcription order
        genome_positions = []
        for part in feature.location.parts:
            start, end = int(part.start), int(part.end)
            if strand >= 0:
                # Forward strand: positions go 5' to 3'
                positions = list(range(start + 1, end + 1))  # +1 for 1-based
            else:
                # Reverse strand: positions go 3' to 5' (reverse order)
                positions = list(range(end, start, -1))  # Already 1-based
            genome_positions.extend(positions)
        
        # Handle codon_start (frame offset)
        frame_offset = info["codon_start"] - 1
        
        # Process each nucleotide
        n = len(cds_seq)
        for i in range(n):
            base = cds_seq[i]
            
            # Calculate codon information
            in_frame = i + frame_offset
            codon_pos = (in_frame % 3) + 1  # 1, 2, or 3
            codon_number = (in_frame // 3) + 1
            
            # Get the full codon (3 bases)
            codon_start_idx = in_frame - (in_frame % 3) - frame_offset
            if codon_start_idx >= 0 and codon_start_idx + 2 < n:
                codon = cds_seq[codon_start_idx:codon_start_idx + 3]
                aa = codon_to_amino_acid(codon, info["transl_table"])
            else:
                codon = "NNN"  # Incomplete codon
                aa = "?"
            
            # Add row to our table
            rows.append({
                "genome_pos": genome_positions[i],
                "gene": info["gene"],
                "product": info["product"],
                "base": base,
                "codon": codon,
                "codon_pos": codon_pos,
                "codon_number": codon_number,
                "aa": aa,
            })
    
    # Convert to DataFrame and sort by position
    df = pd.DataFrame(rows)
    df = df.sort_values(["gene", "product", "genome_pos"])
    df = df.reset_index(drop=True)
    
    return df

print("Building CDS lookup table (this may take a moment)...")
cds_table = build_cds_lookup_table(record)
print(f"Done! Table has {len(cds_table):,} rows")

In [ ]:
# Let's look at the CDS table we built
print("CDS lookup table summary:")
print(f"  Total nucleotide positions: {len(cds_table):,}")
print(f"  Unique genes: {cds_table['gene'].nunique()}")
print(f"  Unique products: {cds_table['product'].nunique()}")

print("\nFirst 15 rows (showing start of first gene):")
cds_table.head(15)

---
## Step 5: Load the Variant Table

The variant table contains mutations found in the samples. Each row is one variant with:
- Position on the genome
- Reference allele (what the genome has)
- Alternate allele (what the mutation is)
- Frequency information

In [ ]:
# Load the variant table
# This file has no header, so we specify column names
variants = pd.read_csv(
    variants_path,
    sep="\t",           # Tab-separated
    header=None,         # No header row
    names=["var_id", "samples", "af_min", "af_mean", "af_max", "REF", "ALT"]
)

print(f"Loaded {len(variants)} variants")
print("\nColumn descriptions:")
print("  var_id: Unique identifier (includes position)")
print("  samples: Number of samples with this variant")
print("  af_min/mean/max: Allele frequency statistics")
print("  REF: Reference allele")
print("  ALT: Alternate allele (the mutation)")

variants.head()

In [ ]:
# Extract the position number from the var_id column
# The var_id looks like "10044GA" - the number is the position

variants["POS"] = variants["var_id"].str.extract(r"^(\d+)").astype(int)

print("Added POS column:")
print(f"  Position range: {variants['POS'].min()} to {variants['POS'].max()}")
variants[["var_id", "POS", "REF", "ALT"]].head()

---
## Step 6: Annotate Variants

Now we'll combine the variants with our CDS lookup table to determine the effect of each mutation.

### The Process

For each variant:
1. Look up the position in the CDS table
2. If not found → it's **intergenic** (not in a gene)
3. If found:
   - Get the original codon and amino acid
   - Create the mutated codon (swap the variant base)
   - Translate the mutated codon
   - If same amino acid → **synonymous**
   - If different → **nonsynonymous**

In [ ]:
# Helper function: get the complement of a DNA base
# This is needed for genes on the reverse strand

COMPLEMENT = {"A": "T", "T": "A", "G": "C", "C": "G",
              "a": "t", "t": "a", "g": "c", "c": "g"}

def complement(base):
    """Return the complement of a DNA base (A↔T, G↔C)."""
    return COMPLEMENT.get(base, base)

print("Testing complement:")
for b in ["A", "T", "G", "C"]:
    print(f"  {b} -> {complement(b)}")

In [ ]:
def annotate_variants_with_effects(variants_df, cds_df):
    """
    Annotate each variant with its effect on the protein.
    
    Returns a new DataFrame with additional columns:
    - strand: '+' or '-' (which strand the gene is on)
    - new_codon: the codon after the mutation
    - new_aa: the amino acid after the mutation
    - effect: 'synonymous', 'nonsynonymous', or 'intergenic'
    """
    
    # Merge variants with CDS table based on position
    # This gives us the codon/amino acid context for each variant
    merged = variants_df.merge(
        cds_df,
        left_on="POS",
        right_on="genome_pos",
        how="left"  # Keep all variants, even if not in a gene
    )
    
    # Lists to store our results
    strands = []
    new_codons = []
    new_aas = []
    effects = []
    
    # Process each row
    for idx, row in merged.iterrows():
        
        # Check if this position is in a gene
        if pd.isna(row.get("codon")):
            # Not in a gene
            strands.append(None)
            new_codons.append(None)
            new_aas.append(None)
            effects.append("intergenic")
            continue
        
        # Get the values we need
        ref_base = str(row["REF"]).upper()
        alt_base = str(row["ALT"]).upper()
        cds_base = str(row["base"]).upper()  # Base in the CDS (may be complemented)
        codon = str(row["codon"]).upper()
        codon_pos = int(row["codon_pos"])  # 1, 2, or 3
        old_aa = str(row["aa"])
        
        # Determine strand by comparing ref_base to cds_base
        if ref_base == cds_base:
            # Same base = forward strand
            strand = "+"
            coding_alt = alt_base
        elif complement(ref_base) == cds_base:
            # Complement = reverse strand
            strand = "-"
            coding_alt = complement(alt_base)
        else:
            # Mismatch - something's wrong
            strands.append("?")
            new_codons.append(None)
            new_aas.append(None)
            effects.append("unknown")
            continue
        
        strands.append(strand)
        
        # Create the mutated codon
        # Replace the base at codon_pos with the new base
        codon_list = list(codon)
        codon_list[codon_pos - 1] = coding_alt  # -1 because codon_pos is 1-based
        new_codon = "".join(codon_list)
        new_codons.append(new_codon)
        
        # Translate the new codon
        new_aa = codon_to_amino_acid(new_codon)
        new_aas.append(new_aa)
        
        # Determine the effect
        if old_aa == "?" or new_aa == "?":
            effects.append("unknown")
        elif new_aa == old_aa:
            effects.append("synonymous")
        else:
            effects.append("nonsynonymous")
    
    # Add our results to the DataFrame
    merged["strand"] = strands
    merged["new_codon"] = new_codons
    merged["new_aa"] = new_aas
    merged["effect"] = effects
    
    return merged

print("Annotating variants (this may take a moment)...")
annotated = annotate_variants_with_effects(variants, cds_table)
print(f"Done! Annotated {len(annotated)} rows")
print("\n(Note: Some variants appear multiple times if they affect overlapping genes)")

---
## Step 7: View Results

Let's see what we found!

In [ ]:
# Summary of effects
print("=" * 50)
print("VARIANT EFFECT SUMMARY")
print("=" * 50)
print(annotated["effect"].value_counts())
print("\n")

# Calculate percentages
total = len(annotated)
for effect in ["nonsynonymous", "synonymous", "intergenic"]:
    count = (annotated["effect"] == effect).sum()
    pct = 100 * count / total
    print(f"  {effect}: {count} ({pct:.1f}%)")

In [ ]:
# View the annotated variants
# Select the most important columns for display

display_columns = [
    "var_id",      # Variant ID
    "POS",         # Genome position
    "REF",         # Reference allele
    "ALT",         # Alternate allele
    "gene",        # Gene name
    "product",     # Protein product
    "codon",       # Original codon
    "new_codon",   # Mutated codon
    "aa",          # Original amino acid
    "new_aa",      # New amino acid
    "effect",      # Effect type
    "samples",     # Number of samples
    "af_mean"      # Mean allele frequency
]

print("First 20 annotated variants:")
annotated[display_columns].head(20)

In [ ]:
# Summary by gene
print("Effect counts by gene:")
gene_summary = annotated.pivot_table(
    index="gene",
    columns="effect",
    values="var_id",
    aggfunc="nunique",  # Count unique variants
    fill_value=0
)
gene_summary

---
## Step 8: Visualization

We'll create two charts:
1. **Variant track**: Shows where variants are located and their effects
2. **Amino acid heatmap**: Shows the amino acid context across genes

Both charts share the same X-axis (genome position), so you can zoom in on one and both will update.

In [ ]:
# Prepare data for visualization

# For the heatmap: add a label column for amino acids
cds_vis = cds_table.copy()
cds_vis["aa_label"] = cds_vis["aa"].replace({"*": "Stop", "?": "Unknown"})

# Get the order of products (genes) by their position on the genome
product_order = (
    cds_vis.groupby("product")["genome_pos"]
    .min()
    .sort_values()
    .index.tolist()
)

print(f"Products (genes) in order: {len(product_order)}")

In [ ]:
# Create the amino acid heatmap
# Each rectangle represents one nucleotide, colored by amino acid

# Add columns for rectangle coordinates
cds_vis["x2"] = cds_vis["genome_pos"] + 1  # End of rectangle

heatmap = alt.Chart(cds_vis).mark_rect().encode(
    # X-axis: genome position
    x=alt.X("genome_pos:Q", title="Genome position"),
    x2="x2:Q",
    
    # Y-axis: gene product
    y=alt.Y("product:N", sort=product_order, title="Gene"),
    
    # Color by amino acid
    color=alt.Color("aa_label:N", title="Amino acid"),
    
    # Tooltip shows details on hover
    tooltip=["gene", "product", "genome_pos", "codon", "aa_label", "base"]
).properties(
    width=900,
    height=max(140, 18 * len(product_order)),
    title="Amino acid context by position"
)

print("Heatmap created!")

In [ ]:
# Create the variant track
# Shows variants as tick marks, colored by effect

# Filter to only coding variants (synonymous or nonsynonymous)
coding_variants = annotated[
    annotated["effect"].isin(["synonymous", "nonsynonymous"])
].copy()

# Calculate allele frequency range for sizing
coding_variants["af_range"] = coding_variants["af_max"] - coding_variants["af_min"]

# Get product order for these variants
var_product_order = (
    coding_variants.groupby("product")["POS"]
    .min()
    .sort_values()
    .index.tolist()
)

variant_track = alt.Chart(coding_variants).mark_tick(thickness=4).encode(
    # X-axis: genome position
    x=alt.X("POS:Q", title="Genome position"),
    
    # Y-axis: gene product
    y=alt.Y("product:N", sort=var_product_order, title="Gene"),
    
    # Color: green for synonymous, red for nonsynonymous
    color=alt.Color(
        "effect:N",
        title="Effect",
        scale=alt.Scale(
            domain=["synonymous", "nonsynonymous"],
            range=["#2ca02c", "#d62728"]  # Green, Red
        )
    ),
    
    # Size: based on allele frequency range
    size=alt.Size("af_range:Q", title="AF range"),
    
    # Opacity: based on number of samples
    opacity=alt.Opacity("samples:Q", title="Samples"),
    
    # Tooltip shows details on hover
    tooltip=[
        "var_id", "POS", "REF", "ALT",
        "gene", "product", "effect",
        "codon", "new_codon", "aa", "new_aa",
        "samples", "af_mean"
    ]
).properties(
    width=900,
    height=max(120, 16 * len(var_product_order)),
    title="Variants by position (green=synonymous, red=nonsynonymous)"
)

print("Variant track created!")

In [ ]:
# Combine both charts with shared X-axis zooming
# Click and drag on either chart to zoom - both will update!

# Create a selection for X-axis zooming
zoom = alt.selection_interval(bind="scales", encodings=["x"])

# Stack the charts vertically
combined_chart = alt.vconcat(
    variant_track,   # Variants on top
    heatmap,         # Amino acids below
    spacing=10       # Gap between charts
).resolve_scale(
    x="shared",          # Share the X-axis between charts
    color="independent"  # Keep colors separate
).add_params(
    zoom  # Add zooming capability
)

print("Combined chart ready!")
print("Tip: Click and drag on the chart to zoom in. Double-click to reset.")

combined_chart

---
## Step 9: Save Results (Optional)

Uncomment the cells below to save the annotated variants back to your Galaxy history.

In [ ]:
# Uncomment to save the annotated variants as a tabular file in Galaxy

# annotated.to_csv("annotated_variants.tsv", sep="\t", index=False)
# await gxy.put("annotated_variants.tsv", output="Annotated Variants", ext="tabular")
# print("Saved annotated variants to Galaxy history!")

---
## Summary

This notebook:
1. Downloaded GenBank annotation and variant data from Galaxy
2. Built a lookup table mapping every nucleotide to its codon and amino acid
3. Annotated each variant with its effect (synonymous, nonsynonymous, or intergenic)
4. Created an interactive visualization showing variant positions and effects

**Key findings:**
- Most variants are nonsynonymous (change the protein)
- Synonymous variants are less common ("silent" mutations)
- The visualization lets you explore which genes have the most variants